In [1]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2

In [2]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
client.set_timeout(20.0)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [3]:
# Spawn ego vehicle
vehicle_bp = bp_lib.find('vehicle.audi.a2') 
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Move spectator behind vehicle to view
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

vehicle.set_autopilot(True) 

In [4]:
net = cv2.dnn.readNetFromONNX('X:\Installs\carla\PythonAPI\workingDirectory\weights/perfect.onnx')
print("Yolo ONNX Model has been loaded")
# we have to specify which gpu or cpu we are using
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
#gpu
#yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

Yolo ONNX Model has been loaded


In [5]:
#Iterate this cell to find desired camera location
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2)) #Change this to move camera
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

time.sleep(0.2)
spectator.set_transform(camera.get_transform())
#camera.destroy()

In [6]:
# Spawn camera
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle

SyntaxError: unexpected EOF while parsing (1800696291.py, line 3)

In [6]:
# Callback stores sensor data in a dictionary for use outside callback                         
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


In [7]:
# Get gamera dimensions and initialise dictionary                       
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'image': np.zeros((image_h, image_w, 4))}

In [8]:
# Start camera recording
camera.listen(lambda image: camera_callback(image, camera_data))

In [20]:
camera_data

{'image': array([[[105, 112, 120, 255],
         [105, 113, 121, 255],
         [106, 113, 122, 255],
         ...,
         [ 90,  95, 104, 255],
         [ 88,  93, 102, 255],
         [ 88,  93, 101, 255]],
 
        [[  0,   0,   0, 255],
         [  0,   0,   0, 255],
         [  0,   0,   0, 255],
         ...,
         [125, 128, 135, 255],
         [124, 128, 134, 255],
         [124, 128, 134, 255]],
 
        [[122, 127, 133, 255],
         [119, 122, 129, 255],
         [115, 119, 126, 255],
         ...,
         [216, 223, 229, 255],
         [217, 223, 229, 255],
         [215, 222, 229, 255]],
 
        ...,
 
        [[ 88,  96, 105, 255],
         [ 88,  96, 105, 255],
         [ 86,  95, 104, 255],
         ...,
         [  0,   0,   0, 255],
         [  0,   0,   0, 255],
         [  0,   0,   0, 255]],
 
        [[ 88,  96, 105, 255],
         [ 90,  98, 106, 255],
         [ 91,  99, 108, 255],
         ...,
         [ 90,  98, 108, 255],
         [ 90,  98, 108, 2

In [ ]:
image = camera.listen().data

# Convert the image data to a numpy array
np_image = np.frombuffer(image, dtype=np.uint8)

# Reshape the numpy array to the image dimensions (height, width, channels)
reshaped_image = np_image.reshape((image_height, image_width, num_channels))

# Process the image
process_image(reshaped_image)

In [21]:
# OpenCV named window for rendering
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)
image_height = 640
image_width = 480
num_channels = 4
# Game loop
while True:
    frame = camera_data['image']
    #np_image = np.frombuffer(frame, dtype=np.uint8)
    #reshaped_image = np_image.reshape((image_height, image_width, num_channels))
    blob = cv2.dnn.blobFromImage(frame, 1/255, (640, 640), (0, 0, 0), True, crop = False)
    net.setInput(blob)
    outs = net.forward()
    # Imshow renders sensor data to display
    cv2.imshow('RGB Camera', camera_data['image'])
    
    # Quit if user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()
cv2.stop()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\layers\convolution_layer.cpp:420: error: (-2:Unspecified error) Number of input channels should be multiple of 3 but got 4 in function 'cv::dnn::ConvolutionLayerImpl::getMemoryShapes'


In [ ]:
def processImage(image):
    i = np.array(image.raw_data)
    i2 = i.reshape((IM_HEIGHT,IM_WIDTH,4))
    i3 = i2[:,:,:3]

    #detected_image = detector(i3)
    #cv2.imshow('OpenCV Interface of detections',detected_image)
    cv2.imshow('OpenCV Interface of img',i3)
    cv2.waitKey(1)
    return i3/255.0

In [ ]:
!pip show opencv-python

In [ ]:
camera.destroy()
vehicle.destroy()